In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [2]:
from CloudDetect.util import read_off, visualise
from CloudDetect.transform import PointCloudSample, Normalise, Tensor
from CloudDetect.models import PointNet
from CloudDetect.dataset import CloudDataset, assign_val_indices
from CloudDetect.util import get_metrics

In [16]:
from tqdm import tqdm

In [3]:
import glob
metadata = pd.read_csv('data/metadata_modelnet40.csv')
all_data_files = glob.glob('**/*.off',recursive=True)
all_data_files = set(['/'.join(x.split('/')[2:]) for x in all_data_files])
metadata = metadata[metadata['object_path'].map(lambda x: x in all_data_files)]

In [4]:
# data loader

In [5]:
def preprocessing(n_sample, norm_how):
    composition = [
            PointCloudSample(n_sample),
            Normalise(norm_how),
            Tensor()
    ]
    return transforms.Compose(composition)

def generate_class_mapper(metadata):
    class_mapper = {x:i for i,x in enumerate(metadata['class'].unique())}
    return class_mapper

In [6]:
metadata_train = metadata[metadata['split'] == 'train']
class_mapper = generate_class_mapper(metadata_train)
metadata_test = metadata[metadata['split'] == 'test']
metadata_train = metadata_train.reset_index(drop = True)
metadata_test = metadata_test.reset_index(drop = True)
assign_val_indices(metadata_train, 'class', 3)

In [14]:
from tqdm import notebook

In [39]:
class training(object):
    def __init__(self, 
                 loss_fn, 
                 optimiser, 
                 model,
                train_loader,
                val_loader):
        self.loss_fn = loss_fn
        self.optimiser = optimiser
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader

    def load_data(self, data):
        X,y = data['data'], data['category']
        X = torch.transpose(X.float(),1,2)
        return X, y
    
    def perform_optimisation(self,X, y):
        self.optimiser.zero_grad()
        model_output = self.model(X)
        loss = self.loss_fn(model_output, y)
        loss.backward()
        optimiser.step() 
        return loss
    
    def reporting(self, batch_print, running_loss):
        last_loss = running_loss / batch_print # average loss per batch
        return last_loss

    def run_epoch(self):
        running_loss = 0
        last_loss = 0
        epoch_index = 0
        tracker = []
        batch_print = 1
        last_loss = -999
        val_loss = -999
        pbar = tqdm(enumerate(self.train_loader), total=len(self.train_loader))
        for i,x in pbar:
            pbar.set_description(f"BATCH TRAIN LOSS {round(last_loss,5)} - VAL LOSS {round(val_loss,5)}")
            # load data
            X,y = self.load_data(x)
            loss = self.perform_optimisation(X, y)
            running_loss += loss.item()
            if i % batch_print == 0 and i!=0:
                last_loss = self.reporting(batch_print, running_loss)
                running_loss = 0
        
                #validate
                all_true_output = []
                all_model_output = []
                for i,x in enumerate(self.val_loader):
                    # load data
                    X,y = self.load_data(x)
                    model_output = self.model(X)
                    all_true_output.append(y)
                    all_model_output.append(model_output)
        
                all_true = torch.concat(all_true_output)
                all_model = torch.concat(all_model_output)
                loss = loss_fn(all_model, all_true)
                val_loss = loss.item()
        
        
        #validate
        all_true_output = []
        all_model_output = []
        for i,x in tqdm(enumerate(self.val_loader), total=len(self.val_loader)):
            # load data
            X,y = self.load_data(x)
            model_output = self.model(X)
            all_true_output.append(y)
            all_model_output.append(model_output)
        
        all_true = torch.concat(all_true_output)
        all_model = torch.concat(all_model_output)
        loss = self.loss_fn(all_model, all_true)
        print(loss)
        classification_output = torch.argmax(torch.exp(all_model),axis = 1)
        results = get_metrics(all_true, classification_output)
        print(results)

In [40]:
# modelling
loss_fn = torch.nn.NLLLoss()
model = PointNet(n_point = 1024, classes = len(class_mapper), segment = False)
optimiser = torch.optim.Adam(model.parameters(), lr=0.0001)

In [41]:
val_idx = 1
metadata_train_val = metadata_train[metadata_train['kfold'] == val_idx]
metadata_train_train = metadata_train[metadata_train['kfold'] != val_idx]

# jsut for quick training
metadata_train_train = metadata_train_train.sample(100)
metadata_train_val = metadata_train_val.sample(100)

# dataset
ROOT = 'data/ModelNet40/'
preprocessor = preprocessing(1024, 'max')
class_mapper = generate_class_mapper(metadata)
cloud_train_dataset = CloudDataset(metadata_train_train, preprocessor, ROOT, class_mapper)
cloud_val_dataset = CloudDataset(metadata_train_val, preprocessor, ROOT, class_mapper)
CloudDataTrainLoader = DataLoader(cloud_train_dataset, batch_size=32, shuffle=True)
CloudDataValLoader = DataLoader(cloud_val_dataset, batch_size=32, shuffle=False)

In [42]:
train_object = training(loss_fn, optimiser, model, CloudDataTrainLoader, CloudDataValLoader)

In [43]:
train_object.run_epoch()

BATCH TRAIN LOSS 3.77654 - VAL LOSS 3.69732: 100%|█| 4/4 [00:42<00:00, 10.71s/it
100%|█████████████████████████████████████████████| 4/4 [00:11<00:00,  2.77s/it]
/opt/miniconda3/envs/main/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


tensor(3.6603, grad_fn=<NllLossBackward0>)
{'f1_score': 0.041588050314465407, 'precision': 0.052210365853658534, 'recall': 0.05729166666666667, 'balanced_acc': 0.06547619047619048}
